My attempt to make CNN able to clasify dogs and cats<br><br>
Important note dataset have been downloaded from kaggle and extracted to train folder ... <br><br>
Lets start with neccessary imports:


In [10]:
import numpy as np
from PIL import Image
import random
import skimage
import tensorflow as tf
import time

Below code to show avaivable device

In [2]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 3396653073226691788, name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 5908987904
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 14827356250583959885
 physical_device_desc: "device: 0, name: GeForce GTX 1060 6GB, pci bus id: 0000:02:00.0, compute capability: 6.1"]

Next we need to make datates of cats and dogs, resize images convert them to numpy arrays and merge

In [13]:
def get_cats(size = 500, start = 0):
    '''
    for example if you want first 50 images of cats resized to 200 x 200 pixels as grayscaled numpy array
    set size = 50 and start = 0
    if you want images from 50 - 100
    set size 50 and start 50
    '''
    cats = []
    for i in range(start, start + size):
        img = Image.open('train/cat.{}.jpg'.format(i))
        img = img.resize(size = (200,200))
        img = skimage.color.rgb2gray(skimage.img_as_float(img))
        cats.append(img)
    return np.array(cats, dtype='float32')

def get_dogs(size = 500, start = 0):
    '''
    for example if you want first 50 images of dogs resized to 200 x 200 pixels as grayscaled numpy array
    set size = 50 and start = 0
    if you want images from 50 - 100
    set size 50 and start 50
    '''
    dogs = []
    for i in range(start, start + size):
        img = Image.open('train/dog.{}.jpg'.format(i))
        img = img.resize(size = (200,200))
        img = skimage.color.rgb2gray(skimage.img_as_float(img))
        dogs.append(img)
    return np.array(dogs, dtype='float32')

def randomizer():
    animalKind = {'cat':[1,0], 'dog':[0,1]}
    catOrDog = random.choice(['cat', 'dog'])
    pictureIndex = random.choice(range(9999))
    img = Image.open('train/{}.{}.jpg'.format(catOrDog, pictureIndex))
    img = img.resize(size = (200,200))
    img = skimage.color.rgb2gray(skimage.img_as_float(img))
    return {'y':animalKind.get(catOrDog), 'x':img}

def random_X_and_Y(size = 1000):
    X = []
    Y = []
    for i in range(size):
        data = randomizer()
        X.append(data.get('x'))
        Y.append(data.get('y'))
    return np.array(X, dtype = 'float32').reshape(-1,200,200,1), np.array(Y, dtype = 'float32')

In [14]:
x, y = random_X_and_Y()

506.0

Now we can make our dataset

In [5]:
cats = get_cats()
dogs = get_dogs()
X = np.concatenate((cats, dogs), axis = 0).reshape(-1,200,200,1)
X.shape

(1000, 200, 200, 1)

Also we need labels, we can do it like cat will be 0 and dog 1

In [8]:
y_cats = np.array([[1,0]] * cats.shape[0])
y_dogs = np.array([[0,1]] * dogs.shape[0])
Y = np.concatenate((y_cats, y_dogs), axis = 0)
Y.shape

(1000, 2)

Now real fun can begin :)<br>
Here are lots of possibilities to tweek, change convolutional filter (one is for horizontal edges another one for vertical edges), number of neurons, activation functions, loss function ...

In [22]:
config = tf.ConfigProto(
        device_count = {'GPU': 0}
    )
tf.reset_default_graph()
X_ = tf.placeholder( dtype=tf.float32, shape=(1000, 200, 200,1))
Y_ = tf.placeholder( dtype=tf.float32, shape=(1000, 2))
bias = tf.Variable(tf.zeros([1024]), name="bias")
kernel_h = np.array([3, 3])
kernel_h = [ [1,2,1], [0,0,0], [-1,-2,-1] ] # horizontal edges convolutional filter
kernel_v = np.array([3, 3])
kernel_v = [ [1,0,1], [2,0,-2], [-1,0,-1] ] # vertical edges convolutional filter
with tf.name_scope('convolution'):
    conv_w_h = tf.constant(kernel_h, dtype=tf.float32, shape=(3, 3,1,1))
    conv_w_v = tf.constant(kernel_v, dtype=tf.float32, shape=(3, 3,1,1))    
    # applying convolutional filter
    output_h = tf.nn.conv2d(input=X_, filter=conv_w_h, strides= [1,2,2,1], padding='SAME')
    # applying convolutional filter
    output_v = tf.nn.conv2d(input=X_, filter=conv_w_v, strides= [1,2,2,1], padding='SAME')
    #merging  both convolutional filter
    output = tf.sqrt(tf.add(tf.square(output_h), tf.square(output_v)))
    pool = tf.layers.max_pooling2d(inputs = output, pool_size=[2, 2], strides=2)
    pool2flat = tf.reshape(pool,[-1, 50*50])
    dense = tf.layers.dense(inputs = pool2flat, units=1024, activation=tf.nn.relu)
    dense2 = tf.layers.dense(inputs = dense, units=512, activation=tf.nn.relu)
    dense3 = tf.layers.dense(inputs = dense2, units=256, activation=tf.nn.relu)
    dense4 = tf.layers.dense(inputs = dense3, units = 64, activation=tf.nn.sigmoid)
    logits = tf.layers.dense(inputs = dense4, units=2, activation = tf.nn.sigmoid)
    cross_entropy = tf.nn.sigmoid_cross_entropy_with_logits(logits = logits,labels=Y_)
    cost = tf.reduce_mean(cross_entropy)
    summary_cost = tf.summary.scalar('cost', cost)
    optimizer = tf.train.AdamOptimizer(learning_rate=0.00001).minimize(cost)
    accuracy, acc_op = tf.metrics.accuracy(labels = Y_, predictions = tf.round(logits))
    summary_accuracy = tf.summary.scalar('accuracy', accuracy)

We can start training our CNN. Hmm there is some difference between tf.sigmoid and tf.nn.sigmoid<br>
to run tensorboard: <strong>tensorboard --logdir=/home/oktogen/Desktop/DeepLearningExperiments/CNN/CatsVsDogs/Logs<strong>

In [23]:
init = tf.global_variables_initializer()
sess = tf.Session(config = config)
train_writer = tf.summary.FileWriter( '/home/oktogen/Desktop/DeepLearningExperiments/CNN/CatsVsDogs/Logs', sess.graph)
saver = tf.train.Saver()
sess.run(init)
sess.run(tf.local_variables_initializer())
print('Session starting {}'.format(time.ctime()))
for i in range(500):
    X, Y = random_X_and_Y()
    sess.run(optimizer, feed_dict={ X_: X, Y_:Y})
    if i % 10 == 0:
        cost_  = sess.run(cost, feed_dict={ X_: X, Y_:Y})
        sess.run([accuracy, acc_op], feed_dict={ X_: X, Y_:Y})
        acc= sess.run(accuracy, feed_dict={ X_: X, Y_:Y})
        sum_acc, sum_cost = sess.run([summary_accuracy, summary_cost], feed_dict={ X_: X, Y_:Y})
        train_writer.add_summary(sum_cost, i)
        train_writer.add_summary(sum_acc, i)
        print('Time: {} Epoch: {} Cost: {} Acc: {}'.format(time.ctime(), i, cost_, acc) )
        saver.save(sess, '/home/oktogen/Desktop/DeepLearningExperiments/CNN/CatsVsDogs/Model/TF',global_step = i)

Session starting Sun Nov 11 19:14:08 2018
Time: Sun Nov 11 19:14:20 2018 Epoch: 0 Cost: 0.7225949764251709 Acc: 0.5040000081062317
Time: Sun Nov 11 19:15:40 2018 Epoch: 10 Cost: 0.7159053087234497 Acc: 0.5024999976158142
Time: Sun Nov 11 19:16:44 2018 Epoch: 20 Cost: 0.7104380130767822 Acc: 0.4938333332538605
Time: Sun Nov 11 19:17:36 2018 Epoch: 30 Cost: 0.7066082954406738 Acc: 0.4950000047683716
Time: Sun Nov 11 19:18:21 2018 Epoch: 40 Cost: 0.7056167125701904 Acc: 0.4959000051021576
Time: Sun Nov 11 19:19:03 2018 Epoch: 50 Cost: 0.6994985342025757 Acc: 0.49658334255218506
Time: Sun Nov 11 19:19:43 2018 Epoch: 60 Cost: 0.7000135779380798 Acc: 0.49707141518592834
Time: Sun Nov 11 19:20:21 2018 Epoch: 70 Cost: 0.6969941854476929 Acc: 0.4974375069141388
Time: Sun Nov 11 19:20:59 2018 Epoch: 80 Cost: 0.6966108083724976 Acc: 0.49772220849990845
Time: Sun Nov 11 19:21:35 2018 Epoch: 90 Cost: 0.6949924826622009 Acc: 0.4979499876499176
Time: Sun Nov 11 19:22:11 2018 Epoch: 100 Cost: 0.694222

In [238]:
#init = tf.global_variables_initializer()
X, Y = random_X_and_Y()
res = sess.run(logits,feed_dict={ X_: X})


In [ ]:
for i in range(200000, 250000):
    X, Y = random_X_and_Y()
    sess.run(optimizer, feed_dict={ X_: X, Y_:Y})
    if i % 500 == 0:
        cost_  = sess.run(cost, feed_dict={ X_: X, Y_:Y})
        sess.run([accuracy, acc_op], feed_dict={ X_: X, Y_:Y})
        acc= sess.run(accuracy, feed_dict={ X_: X, Y_:Y})
        sum_acc, sum_cost = sess.run([summary_accuracy, summary_cost], feed_dict={ X_: X, Y_:Y})
        train_writer.add_summary(sum_cost, i)
        train_writer.add_summary(sum_acc, i)
        print('Time: {} Epoch: {} Cost: {} Acc: {}'.format(time.ctime(), i, cost_, acc) )
        saver.save(sess, '/home/oktogen/Desktop/DeepLearningExperiments/CNN/CatsVsDogs/Model/TF',global_step = i)

Time: Tue Nov 20 07:00:23 2018 Epoch: 200000 Cost: 0.5237043499946594 Acc: 0.9106477499008179
Time: Tue Nov 20 07:28:16 2018 Epoch: 200500 Cost: 0.5212043523788452 Acc: 0.910766065120697
Time: Tue Nov 20 07:56:15 2018 Epoch: 201000 Cost: 0.5202043652534485 Acc: 0.9108882546424866
Time: Tue Nov 20 08:24:08 2018 Epoch: 201500 Cost: 0.5272043347358704 Acc: 0.9109790325164795
Time: Tue Nov 20 08:52:03 2018 Epoch: 202000 Cost: 0.5267044305801392 Acc: 0.9110715985298157
Time: Tue Nov 20 09:20:19 2018 Epoch: 202500 Cost: 0.5262043476104736 Acc: 0.9111659526824951
Time: Tue Nov 20 09:48:31 2018 Epoch: 203000 Cost: 0.5247043967247009 Acc: 0.9112664461135864
Time: Tue Nov 20 10:16:43 2018 Epoch: 203500 Cost: 0.5217044353485107 Acc: 0.9113796353340149
Time: Tue Nov 20 10:44:56 2018 Epoch: 204000 Cost: 0.5162043571472168 Acc: 0.9115163683891296
Time: Tue Nov 20 11:13:08 2018 Epoch: 204500 Cost: 0.5217043161392212 Acc: 0.9116285443305969
Time: Tue Nov 20 11:41:19 2018 Epoch: 205000 Cost: 0.52670437